# Исследование текущего уровня NPS

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых.

## Подключение к базе

In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных

In [3]:
query = """
SELECT u.user_id AS user_id,
       u.lt_day AS lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'Новый клиент'
           WHEN u.lt_day > 365 THEN 'Старый клиент'
           ELSE 'Не указано'
       END AS is_new,
       u.age AS age,
       CASE
           WHEN u.gender_segment = 1 THEN 'Женщина'
           WHEN u.gender_segment = 0 THEN 'Мужчина'
           ELSE 'Не указано'
       END AS gender_segment,
       u.os_name AS os_name,
       u.cpe_type_name AS cpe_type_name,
       l.country AS country,
       l.city AS city,
       ags.title AS age_segment,
       trs.title AS traffic_segment,
       lts.title AS lifetime_segment,
       u.nps_score AS nps_score,
       CASE 
           WHEN u.nps_score <= 6 AND u.nps_score >= 0 THEN 'Критики'
           WHEN u.nps_score <= 8 AND u.nps_score >= 7 THEN 'Нейтралы'
           WHEN u.nps_score <= 10 AND u.nps_score >= 9 THEN 'Сторонники'
           ELSE 'Не указано'
       END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
LEFT JOIN traffic_segment AS trs ON u.tr_gr_id = trs.tr_gr_id
LEFT JOIN lifetime_segment AS lts ON u.lt_gr_id = lts.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый клиент,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,Сторонники
1,A001WF,2344,Старый клиент,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,Сторонники
2,A003Q7,467,Старый клиент,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,Сторонники


In [5]:
#df.to_csv('telecomm_csi_tableau.csv', index=False)

## Анализ данных в Tableau

https://public.tableau.com/app/profile/artem.kudrin/viz/Combinedproject2/sheet11